<a href="https://colab.research.google.com/github/muyuuuu/colab/blob/main/adversarial-denoise/adp(ICML%202021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4
sigma = 0.1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class Score_Net(nn.Module):
    def __init__(self):
        super(Score_Net, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
# 			nn.Conv2d(48, 96, 4, stride=2, padding=1),           # [batch, 96, 2, 2]
#             nn.ReLU(),
        )
        self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
#             nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [8]:
net = Score_Net().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, _ = data
        a = torch.normal(0, sigma, (batch_size, 3, 32, 32))
        inputs, a = inputs.to(device), a.to(device) 
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs + a)

        loss = criterion(a + sigma**2 * outputs, inputs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.266
[1,  4000] loss: 0.265
[1,  6000] loss: 0.267
[1,  8000] loss: 0.268
[1, 10000] loss: 0.268
[1, 12000] loss: 0.266
[2,  2000] loss: 0.266
[2,  4000] loss: 0.267
[2,  6000] loss: 0.267
[2,  8000] loss: 0.266
[2, 10000] loss: 0.268
[2, 12000] loss: 0.267
Finished Training


In [9]:
torch.save(net, 'score.pth')

In [23]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()
model = model.to(device)

In [52]:
delta = 0.00001
lambda_ = 0.05
tao = 0.001
alpha = None
correct = 0
total = 0


with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        predicted = torch.zeros((10, 4))
        for i in range (10):
            a = torch.normal(0, sigma, (batch_size, 3, 32, 32)).to(device)
            inputs = inputs.to(device)
            inputs += a
            for _ in range (10):
                inputs_ = inputs + delta * net(inputs)
                alpha = lambda_ * delta / (1 - ( torch.transpose(net(inputs), 2, 3) * net(inputs_) / torch.square(net(inputs) )))
                inputs = inputs + alpha * net(inputs)
            # 预测 inputs
            outputs = model(inputs)
            _, idx = torch.max(outputs.data, 1)
            predicted[i] = idx

        # predicted, _ = torch.max(predicted, 0)
        predicted, _ = torch.mode(predicted, 0)
        # break
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %


In [45]:
a = torch.randint(10, (10, 4,))
a

tensor([[8, 2, 0, 7],
        [2, 4, 9, 7],
        [1, 1, 0, 6],
        [9, 9, 8, 9],
        [1, 2, 2, 2],
        [5, 5, 8, 2],
        [8, 1, 5, 9],
        [8, 9, 9, 7],
        [3, 6, 2, 2],
        [1, 1, 5, 8]])

In [46]:
b = torch.mode(a, dim=0)

In [47]:
b

torch.return_types.mode(values=tensor([1, 1, 0, 2]), indices=tensor([9, 9, 2, 8]))